### Setup

#### Imports

In [13]:
import pandas as pd
import os

#### Paths/Folders

In [14]:
data_folder = os.path.join(os.getcwd(), "data/original")

cleaned_data_folder = os.path.join(os.getcwd(), "data/clean")

if not os.path.exists(cleaned_data_folder):
    os.mkdir(cleaned_data_folder)

#### Helper Functions

In [15]:
def load(filename):
    csv = os.path.join(data_folder, filename)
    return pd.read_csv(csv)

def save(dataframe, filename):
    csv = os.path.join(cleaned_data_folder, filename)
    dataframe.to_csv(csv, index=False)

### Data Cleaning

#### Load Factor Data

In [16]:
df = load("load_factor_data.csv")

# Rename organization_code_iata to airline_iata, period_end_date to date, and passenger_load_factor_data_pct to load_factor_data
df = df.rename(
    columns={
        "organization_code_iata": "airline_iata",
        "period_end_date": "date",
        "passenger_load_factor_data_pct": "load_factor_data",
    }
)

# Drop organization_code_icao and period_start_date
df = df.drop(
    columns=["organization_code_icao", "period_start_date"]
)

# Keep rows where accounting_period is month {number}
df = df[
    df["accounting_period"].str.contains("Month")
]

# Drop accounting_period
df = df.drop(columns=["accounting_period"])

# Convert dates to yyyy-mm format
df["date"] = pd.to_datetime(df["date"]).dt.strftime("%Y-%m")

save(df, "load_factor_data.csv")

df.head()

,airline_iata,date,passenger_load_factor_pct
0,EI,2020-07,24.696155
1,EI,2020-08,29.712449
2,EI,2020-09,25.965699
3,EI,2020-10,17.795881
4,EI,2020-11,15.954516


#### Submission

In [17]:
df = load("submission.csv")

# Combine load_year and load_month into new date column at the front in yyyy-mm format
df.insert(
    0,
    "date",
    pd.to_datetime(
        df["load_year"].astype(str) + "-" + df["load_month"].astype(str)
    ).dt.strftime("%Y-%m"),
)

# Drop load_year and load_month
df = df.drop(columns=["load_year", "load_month"])

# Rename organization_code_iata to airline_iata
df = df.rename(
    columns={"organization_code_iata": "airline_iata"}
)

# Rename forecast_load_factor_pct to load_factor
df = df.rename(
    columns={"forecast_load_factor_pct": "load_factor"}
)

save(df, "submission.csv")

df.head()

,date,airline_iata,load_factor
0,2022-01,G4,NaN
1,2022-02,G4,NaN
2,2022-03,G4,NaN
3,2022-04,G4,NaN
4,2022-05,G4,NaN


#### Country

In [18]:
df = load("country.csv")

# Rename country to country_name, iso to country_iso, iata to country_iata, and numeric to country_numeric
df = df.rename(
    columns={
        "country": "country_name",
        "iso": "country_iso",
        "iata": "country_iata",
        "numeric": "country_numeric"
    }
)

save(df, "country.csv")

df.head()

,country_name,country_iata,country_iso,country_numeric
0,Afghanistan,AF,AFG,4
1,Åland Islands,AX,ALA,248
2,Albania,AL,ALB,8
3,Algeria,DZ,DZA,12
4,American Samoa,AS,ASM,16


#### GDP

In [19]:
df = load("gdp.csv")

# Rename location_iso to country_iso
df = df.rename(columns={"location_iso": "country_iso"})

# Drop indicator and subject
df = df.drop(columns=["indicator", "subject"])

# Drop rows where frequency is annual
df = df[df["frequency"] != "A"]

# Drop frequency
df = df.drop(columns=["frequency"])

# Rename time to date and value to gdp
df = df.rename(columns={"time": "date", "value": "gdp"})

# Convert dates to yyyy-mm format
df = df.assign(date=pd.to_datetime(df["date"]).dt.strftime("%Y-%m"))

# Drop rows where measure is PC_CHGPY
df = df[df["measure"] != "PC_CHGPY"]

# Drop measure
df = df.drop(columns=["measure"])

# Create new rows for each month in date column
df = df.assign(
    date=pd.to_datetime(df["date"])
    .dt.to_period("M")
    .apply(lambda r: r.to_timestamp("M"))
    .dt.strftime("%Y-%m")
)

save(df, "gdp.csv")

df.head()

,country_iso,date,gdp
47,KOR,2017-01,0.984785
48,KOR,2017-04,0.714648
49,KOR,2017-07,1.441838
50,KOR,2017-10,-0.319869
51,KOR,2018-01,1.205852


#### Seats

In [20]:
df = load("seats.csv")

# Rename operating_airline to airline_iata and total_seats to seats
df = df.rename(columns={"operating_airline": "airline_iata", "total_seats": "seats"})

# Combine seats_year and seats_month into new date column before total_seats column in yyyy-mm format
df.insert(
    3,
    "date",
    pd.to_datetime(
        df["seats_year"].astype(str) + "-" + df["dseats_month"].astype(str)
    ).dt.strftime("%Y-%m"),
)

# Drop seats_year and seats_month
df = df.drop(columns=["seats_year", "seats_month"])

save(df, "df.csv")

df.head()

,airline_iata,departure_country_iata,arrival_country_iata,date,seats
0,8E,US,US,2017-10,3130
1,AA,US,US,2021-12,17677677
2,BB,VI,VI,2020-03,2295
3,NH,JP,JP,2020-08,3146684
4,NX,CN,MO,2022-02,58849


#### Validation

In [21]:
df = load("validation.csv")

# Drop load_year, load_month, and organization_code_icao
df = df.drop(columns=["load_year", "load_month", "organization_code_icao"])

# Rename time to date, organization_code_iata to airline_iata, and passenger_load_factor_pct to load_factor
df = df.rename(
    columns={
        "time": "date",
        "organization_code_iata": "airline_iata",
        "passenger_load_factor_pct": "load_factor",
    }
)

# Convert date to yyyy-mm format
df["date"] = pd.to_datetime(df["date"]).dt.strftime("%Y-%m")

save(df, "df.csv")

df.head()

,date,airline_iata,load_factor
0,2022-01,5T,58.690042
1,2022-02,5T,63.308469
2,2022-03,5T,64.637411
3,2022-04,5T,66.932459
4,2022-05,5T,67.446506
